# Simple 3D cantilever beam example 

This simple tutorial show the basic principle to be able to launch mechanical simulation using FEDOO on a very simple example. 

The beam have a square section and the dimensions : 
- Lenght = 1000mm along the $\vec{x}$ axis, 
- Side lenght = 100mm.

The beam is clamped on the left side in the section defined by $x=0$ and a displacement along $\vec{y}$ is enforced on the right side defined by $x=1000$.

### Import library

In [1]:
from fedoo import * #Import all the fedoo library
import numpy as np 
import time #to evaluate the computational time

### Dimension of the problem

The first step is to defined the dimension of the problem. For the simple 3D cantilever beam problem, the dimension of the problem is "3D".

In [2]:
Util.ProblemDimension("3D")


Dimension of the problem is now set on 3D


### Geometry and Mesh creation 
Create a simple 3D mesh using the BoxMesh method of the Mesh Object.
The mesh named 'Domain' is created using $Nx = 31$ nodes along $\vec{x}$, $Ny = Nz = 21$ nodes along $\vec{y}$ and $\vec{z}$.
The element shape used for the geometric interpolation is 'hex8' (8 nodes hexaedral element).

In [3]:
meshID = "Domain"
Mesh.BoxMesh(Nx=31, Ny=21, Nz=21, x_min=0, x_max=1000, 
             y_min=0, y_max=100, z_min=0, z_max=100, 
             ElementShape = 'hex8', ID = meshID)

## Material definition
For mechanical simulation, a material constitutive law have to be defined associated to an ID.
In this simple problem, an elastic isotropic law named 'ElasticLaw' is defined with:

- Young modulus: $E=200 \times 10^3$ MPa
- Poisson ratio: $\nu = 0.3$

A weak formulation of the internal force problem (mechanical equilibrium equation) is then defined.
The internal force weak formulation without prestress and initial displacement is
$$ \int_\Omega \varepsilon(u^\star) D \varepsilon(u) d\Omega = 0 $$
where $D$ is given by the constitutive law.

By default, the ID of the weak form is the same as the ID of the constitutive law, ie 'ElasticLaw'. 

In [4]:
ConstitutiveLaw.ElasticIsotrop(200e3, 0.3, ID = 'ElasticLaw')
WeakForm.InternalForce("ElasticLaw")

### Matrix assembly
A global matrix assembly is created based on the weakform 'ElasticLaw' and with the ID 'assembling'. The global stiffness matrix is then computed. This matrix can be extracted from the assembly using the function GetMatrix.

In [5]:
Assembly.Create("ElasticLaw", meshID, 'hex8', ID="Assembling") 
M = Assembly.GetAll()['Assembling'].GetMatrix()

### Definition of the problem and boundary conditions
A static problem is then defined, ie we want to solve: 
$$ K u = F $$
where $K$ is the stiffness matrix computed before and $F$ is the force vector related to external load. 

In [6]:
Problem.Static("Assembling")

<fedoo.libProblem.Problem_Static.Static.<locals>.__Static at 0x1613e80a518>

The boundary conditions are included directly to the problem. 
We began by extrating the node set corresponding to the left and right side of the mesh.  

In [8]:
nodes_left = Mesh.GetAll()[meshID].GetSetOfNodes("left")
nodes_right = Mesh.GetAll()[meshID].GetSetOfNodes("right")
nodes_top = Mesh.GetAll()[meshID].GetSetOfNodes("top")
nodes_bottom = Mesh.GetAll()[meshID].GetSetOfNodes("bottom")

Application of boundary condition using the BoundaryCondition method. 'Dirichlet' boundary condition (ie displacement boundary conditions) are used. The alternative is 'Neumann' boundary conditions for external force.
It is important to use the method 'ApplyBoundaryCondition()' to include the defined boundary condition in the problem. 

In [9]:
Problem.BoundaryCondition('Dirichlet','DispX',0,nodes_left)
Problem.BoundaryCondition('Dirichlet','DispY', 0,nodes_left)
Problem.BoundaryCondition('Dirichlet','DispZ', 0,nodes_left)
Problem.BoundaryCondition('Dirichlet','DispY', -10, nodes_right)
Problem.ApplyBoundaryCondition()

### Problem resolution
Here the conjugate gradient method is used (solver 'CG')

In [10]:
t0 = time.time() 
Problem.SetSolver('cg')
print('Solving...')
Problem.Solve() 
print('Done in ' +str(time.time()-t0) + ' seconds')

Solving...
Done in 2.155747413635254 seconds


### Post-treatment
Write a vtk file with:
- The displacement vector
- The strain tensor (nodal values)
- The stress tensor (nodal values)
- The vonMises stress

Then, print the total amount of stored elastic energy.

In [11]:
#Get the displacement vector on nodes for export to vtk
U = np.reshape(Problem.GetDoFSolution('all'),(3,-1)).T

#Get Strain and stress tensor
TensorStrain = Assembly.GetAll()['Assembling'].GetStrainTensor(Problem.GetDisp(), "Nodal")       
TensorStress = ConstitutiveLaw.GetAll()['ElasticLaw'].GetStress(TensorStrain)
                                                 
#Write the vtk file                            
OUT = Util.ExportData(meshID)

OUT.addNodeData(U.astype(float),'Displacement')
OUT.addNodeData(TensorStress.vtkFormat(),'Stress')
OUT.addNodeData(TensorStrain.vtkFormat(),'Strain')
OUT.addNodeData(TensorStress.vonMises(),'VMStress')

OUT.toVTK("simple_cantilever_3D_model.vtk")

print('Elastic Energy: ' + str(Problem.GetElasticEnergy()))
print('Result file "simple_cantilever_3D_model.vtk" written in the active directory')

NameError: name 'np' is not defined